<div style="background-color: #FFFFFF;">
    <div style="color:#0000FF; font-size:18px; font-weight: bold;">Librer&iacute;as que deben instalarse previamente:</div>
    <div style="font-family: 'Lucida Console'; background-color: #455A64; color: #ffffff;">
    [python_path]$ python -m pip install --upgrade pip<br>
    [python_path]$ python -m pip install numpy<br>
    [python_path]$ python -m pip install scikit-image<br>
    [python_path]$ python -m pip install pillow
    </div>
</div>

<span style="color:#0000FF; font-size:18px; font-weight: bold;">1. Se importan las librerías necesarias:</span>

In [9]:
import os
import numpy as np
from PIL import Image
import skimage.filters as skf
import skimage.morphology as skm
import mahotas as mht

<div style="color:#0000FF; font-size:18px; font-weight: bold;">2. M&eacute;todo de procesamiento de la imagen para realzar, segmentar y extraer el esqueleto de las arterias.</div>
<div>    
<table style="border-style:hidden; border-collapse:collapse; margin-left:16px;">
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Par&aacute;metros:</td>
    </tr>    
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap;" colspan="">
            img_path :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            La ruta absoluta donde se encuentra la imagen que se va a procesar.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Retorno:</td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            img_arr_int :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Una matriz de numpy con valores enteros en el rango [0, 255] que representa
            la imagen en escala de niveles de gris.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            img_arr_norm :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Una matriz de numpy con valores en el rango [0, 1] que representa
            la imagen en escala de niveles de gris.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            img_vessel_raw :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Una matriz de numpy con valores en el rango [0, 1] que representa
            la respuesta del m&eacute;todo de Frangi, aplicado para el realzado arterial.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            img_vessel_bin :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Una matriz de numpy con valores discretos [False, True] (imagen binaria)
            que representa la imagen con la segmentaci&oacute;n arterial.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            img_vessel_skel :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Una matriz de numpy con valores discretos [False, True] (imagen binaria)
            que contiene el esqueleto arterial.
        </td>
    </tr>
</table>
</div>

In [10]:
def procesar_imagen(img_path):
    # Se carga la imagen:
    img = Image.open(img_path)    
    # Se traslada a una matriz:
    img_arr_int = np.asarray(img.getdata(), dtype=np.uint8).reshape(img.size[0], img.size[1])
    # Se debe normalizar la imagen:
    img_arr_norm = img_arr_int / 255.0
    
    # Se realzan las arterias con el metodo de Frangi:
    sigmas = np.arange(1, 12, 0.5)
    beta_one = 0.5
    beta_two = 15
    img_vessel_raw = skf.frangi(img_arr_norm, sigmas=sigmas, alpha=beta_one, beta=beta_two, mode='wrap')
    
    # Se segmentan las arterias binarizando la imagen
    # con el método de Otsu:
    threshold = skf.threshold_otsu(img_vessel_raw)
    img_vessel_bin = img_vessel_raw > threshold
    
    # Se extrae el esqueleto arterial:
    img_vessel_skel = skm.skeletonize(img_vessel_bin)
    
    # Se devuelven las imagenes procesadas:
    return img_arr_int, img_arr_norm, img_vessel_raw, img_vessel_bin, img_vessel_skel

<div style="color:#0000FF; font-size:18px; font-weight: bold;">3. M&eacute;todo para extraer caracteristicas de Intensidad</div>
<div>    
<table style="border-style:hidden; border-collapse:collapse; margin-left:16px;">
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Par&aacute;metros:</td>
    </tr>    
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap;" colspan="">
            img_arr_norm :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Una matriz de numpy con valores en el rango [0, 1] que representa
            la imagen en escala de niveles de gris.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Retorno:</td>
    </tr>    
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden;   border-collapse:collapse; vertical-align:top;">
            intensity_feats :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Un arreglo numpy con 6 valores que corresponden a las
            caracter&iacute;sticas de intensidad:
            <ol>
                <li>M&iacute;nimo</li>
                <li>M&aacute;ximo</li>
                <li>Mediana</li>
                <li>Promedio</li>
                <li>Varianza</li>
                <li>Desviaci&oacute;n Est&aacute;ndar</li>
            </ol>
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            feats_name :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Un arreglo numpy con los nombres de las caracter&iacute;sticas.
        </td>
    </tr>
</table>
</div>

In [11]:
def extraer_intensidad(img_arr_norm):
    pix_min = np.min(img_arr_norm)
    pix_max = np.max(img_arr_norm)
    pix_median = np.median(img_arr_norm)
    pix_mean = np.mean(img_arr_norm)
    pix_variance = np.var(img_arr_norm)
    pix_stddev = np.std(img_arr_norm)
    
    intensity_feats=np.array([
                        pix_min.item(), pix_max.item(), pix_median.item(),
                        pix_mean.item(), pix_variance.item(), pix_stddev.item()
                    ])
    feats_name =    [
                        'Min', 'Max', 'Median', 
                        'Mean', 'Variance', 'StandardDeviation'
                    ]
    
    return intensity_feats, feats_name

<div style="color:#0000FF; font-size:18px; font-weight: bold;">4. M&eacute;todo para extraer caracteristicas de Textura</div>
<div>    
<table style="border-style:hidden; border-collapse:collapse; margin-left:16px;">
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Par&aacute;metros:</td>
    </tr>    
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap;" colspan="">
            img_arr_norm :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Una matriz de numpy con valores en el rango [0, 1] que representa
            la imagen en escala de niveles de gris.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Retorno:</td>
    </tr>    
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            text_feats :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Un arreglo numpy con 13 valores que corresponden a las primeras 13 caracter&iacute;sticas de textura de Haralik.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            feats_name :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Un arreglo numpy con los nombres de las 13 caracter&iacute;sticas:            
        </td>
    </tr>
</table>
</div>

In [12]:
def extraer_textura(img_arr_int):
    text_feats = mht.features.haralick(img_arr_int, return_mean=True)
    feats_name =[
                    'AngularSecondMoment',
                    'Contrast',
                    'Correlation',
                    'Variance',
                    'Homogeneity',
                    'SumAverage',
                    'SumVariance',
                    'SumEntropy',
                    'Entropy',
                    'DifferenceVariance',
                    'DifferenceEntropy',
                    'InformationMeasureOfCorrelation1',
                    'InformationMeasureOfCorrelation2'
                ]
    return text_feats, feats_name

<div style="color:#0000FF; font-size:18px; font-weight: bold;">5. M&eacute;todo para extraer caracteristicas Morfol&oacute;gicas</div>
<div>    
<table style="border-style:hidden; border-collapse:collapse; margin-left:16px;">
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Par&aacute;metros:</td>
    </tr>    
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            img_arr_norm :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Una matriz de numpy con valores en el rango [0, 1] que representa
            la imagen en escala de niveles de gris.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            img_vessel_bin :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Una matriz numpy con valores l&oacute;gicos que representan la imagen binaria
            con la segmentaci&oacute;n arterial.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            img_vessel_skel :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Una matriz numpy con valores l&oacute;gicos que representan la imagen binaria
            del esqueleto arterial.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Retorno:</td>
    </tr>    
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse; vertical-align:top;">
            morph_feats :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Un arreglo numpy con 4 valores que corresponden a las caracter&iacute;sticas 
            morfol&oacute;gicas extra&iacute;das.<br>
            <ol>
                <li>Cantidad de pixeles arteriales</li>
                <li>Densidad de pixeles arteriales</li>
                <li>Suma de las longitudes de los segmentos arteriales</li>
                <li>Coeficiente de variaci&oacute;n de gris de los segmentos arteriales</li>
            </ol>
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;">
            feats_name :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Un arreglo numpy con los nombres de las 4 caracter&iacute;sticas
            morfol&oacute;gicas extra&iacute;das:
        </td>
    </tr>
</table>
</div>

In [13]:
def extraer_morfologia(img_arr_norm, img_vessel_bin, img_vessel_skel):
    number_of_vessel_pixels = np.sum(img_vessel_bin)
    vessel_density = np.sum(img_vessel_bin*1.0) / (img_vessel_bin.shape[0] * img_vessel_bin.shape[1])
    sum_of_vessels_length = np.sum(img_vessel_skel)
    
    vessel_mean_grays = np.mean(img_arr_norm[img_vessel_bin > 0])
    gray_level_coefficient_variation = np.std(img_arr_norm[img_vessel_bin > 0]) / vessel_mean_grays if vessel_mean_grays > 0 else 0
    
    morph_feats=np.array([
                    number_of_vessel_pixels.item(),
                    vessel_density.item(),
                    sum_of_vessels_length.item(),
                    gray_level_coefficient_variation.item()
                ])
    feats_name =[
                    'NumberOfVesselPixels',
                    'VesselDensity',
                    'SumOfVesselsLength',
                    'GrayLevelCoefficientVariation'
                ]
    
    return morph_feats, feats_name

<div style="color:#0000FF; font-size:18px; font-weight: bold;">6. M&eacute;todo para extraer todas las caracter&iacute;sticas y generar el dataset:</div>
<div>    
<table style="border-style:hidden; border-collapse:collapse; margin-left:16px;">
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Par&aacute;metros:</td>
    </tr>    
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            img_arr_norm :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Una matriz de numpy con valores en el rango [0, 1] que representa
            la imagen en escala de niveles de gris.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse; color: #4E342E; font-weight: bold;" colspan="3">Retorno:</td>
    </tr>    
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;">
            features_dataset :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            El dataset generado como una matriz de numpy en donde, cada rengl&oacute;n corresponde a una imagen y cada columna representa
            cada una de las caracter&iacute;sticas extra&iacute;das.
        </td>
    </tr>
    <tr>
        <td style="border-style:hidden; border-collapse:collapse;"></td>
        <td style="color: #2E7D32; font-weight: bold; width:auto; white-space: nowrap; border-style:hidden; border-collapse:collapse;" colspan="">
            headers :
        </td>
        <td style="border-style:hidden; border-collapse:collapse;">
            Un arreglo de numpy (vector) con los nombres de las columnas asociadas con el
            dataset generado.
        </td>
    </tr>
</table>
</div>

In [14]:
def extraer_todo(ruta:str):
    headers = []
    features_dataset = None
    
    # Se enlistan los archivos dentro del directorio:
    files = os.listdir(ruta)
    
    # Se itera sobre cada elemento que apunta a un archivo:
    for i in range(0, len(files)):
        # Se establece la ruta completa del archivo:
        f = os.path.join(ruta, files[i])
        
        print("Procesando Archivo %d de %d: [%s]" % ((i+1), len(files), f))
        
        # Se procesa la imagen para hacer el realzado arterial, segmentar las arterias y extraer el esqueleto:
        img_arr_int, img_arr_norm, img_vessel_raw, img_vessel_bin, img_vessel_skel = procesar_imagen(f)
        
        # Se extraen las caracteristicas de intensidad, textura y morfologia: 
        intensity_feats, intens_feats_name = extraer_intensidad(img_arr_norm)
        text_feats, text_feats_name = extraer_textura(img_arr_int)
        morph_feats, morph_feats_name = extraer_morfologia(img_arr_norm, img_vessel_bin, img_vessel_skel)
        
        # Se genera un solo vector de caracteristicas con los vectores obtenidos previamente:
        feat_vector = np.hstack((intensity_feats, text_feats, morph_feats))
        
        # Se va generando el dataset. La primera vez, se genera el vector de encabezados.
        if i == 0:
            headers = intens_feats_name + text_feats_name + morph_feats_name
            features_dataset = feat_vector
        else :
            features_dataset = np.vstack((features_dataset, feat_vector))
    return features_dataset, headers

<div style="color:#0000FF; font-size:18px; font-weight: bold;">7. M&eacute;todo principal en donde se define el directorio de trabajo y se invoca el m&eacute;todo que extrae todas las caracter&iacute;sticas.</div>

In [ ]:
def __main__():
    # Se define el directorio que contiene las imagenes:    
    #dir_working     = 'D:/xampp/htdocs/mgilr/ece2024/contents/WorkSpaceECE2024/DatasetECE2024/'
    dir_working     = '/opt/lampp/htdocs/public/mgilr/ece2024/contents/WorkSpaceECE2024/DatasetECE2024/'
    dir_training    = dir_working + '01_Training/'
    dir_validation  = dir_working + '02_Validation/'
        
    # Se extraen las caracteristicas de las imagenes que se utilizaran
    # para el entrenamiento y se generan el dataset y los encabezados 
    # de las columnas:
    dataset_training, headers_training      = extraer_todo(dir_training)
    
    # Se extraen las caracteristicas de las imagenes que se utilizaran
    # para validacion y se generan el dataset y los encabezados 
    # de las columnas:
    dataset_validation, headers_validation  = extraer_todo(dir_validation)    
    
    # Se genera un encabezado para los archivos csv de entrenamiento y validacion:
    csv_header = ','.join(headers_training)
    
    # Se guardan los datasets como archivos de texto separados por coma:
    np.savetxt(dir_working + '01_Training.csv', dataset_training, delimiter=',', fmt='%f', header=csv_header, comments='')
    np.savetxt(dir_working + '02_Validation.csv', dataset_validation, delimiter=',', fmt='%f', header=csv_header, comments='')
    
    print('Data Files saved')

__main__()